**Tibor Swaption using Hull White model**
- [QuantLib - Calibrating Hull White one-factor on negative interest rates](https://quant.stackexchange.com/questions/59269/quantlib-calibrating-hull-white-one-factor-on-negative-interest-rates?rq=1)
- [bermudan-swaption in QuantLib-SWIG](https://github.com/lballabio/QuantLib-SWIG/blob/master/Python/examples/bermudan-swaption.py) 

Tibor curve at Aug19, 2022  
<img src='TiborCurve08192022.png' width='600'>

1. イールドカーブ、ボラティリティデータ 設定

In [1]:
import QuantLib as ql ; import pandas as pd
import myUtil as mu ; from myABBR import *
tradeDT = mu.jDT(2022,8,19) ; mu.setEvDT(tradeDT)
# Tiborカーブ準備
crvDATA = [ ('depo','6m',0.13636),  ('swap','1y',0.15249),  ('swap','18m',0.18742), 
            ('swap','2y',0.20541),  ('swap','3y',0.23156),  ('swap','4y',0.25653), 
            ('swap','5y',0.28528),  ('swap','6y',0.32341),  ('swap','7y',0.36591), 
            ('swap','8y',0.40906),  ('swap','9y',0.45471),  ('swap','10y',0.50224)] 
tbrIX, tbCrvOBJ, tbCrvHDL, tbParRATE = mu.makeTiborCurve(crvDATA)
# check
nodesDF = pd.DataFrame([(dt.ISO(),df) for dt,df in tbCrvOBJ.nodes()], 
                                                                columns=['','DF'])
print("決済日(reference): ", tbCrvOBJ.referenceDate().ISO())
nodesDF[:5].style.format({'DF':'{:.9f}' })

決済日(reference):  2022-08-23


,,DF
0,2022-08-23,1.000000000
1,2023-02-24,0.999309338
2,2023-08-23,0.998476779
3,2024-02-26,0.997170374
4,2024-08-23,0.995895547


2. カリブレーション ( Hull Whiteモデル + Jamshidianエンジンの指定 )

- hwMDLオブジェクトのcalibrateメソッドの仕様(Methods inherited from "CalibratedModel")
<img src='qlr-calibrate.png' width='550'>  
 
- SwaptionHelperの仕様でnormalボラの場合 errortype以降を指定する必要がある  
<img src='SwaptionHelper-const.png' width='550'>

- **カリブのエンジンはjamENG = ql.JamshidianSwaptionEngine(hwMDL)**  
 (21行目)カリブ後 hwMDLのパラメータparams()は上書き
- **バミューダンのエンジンは treeENG = ql.TreeSwaptionEngine(hwMDL, timeGRD)**

In [2]:
#モデル設定 (Hull-Whiteモデル+Jamshidianトリック)
aa,sigma = 0.03, 0.10/100 
hwMDL    = ql.HullWhite(tbCrvHDL, aa, sigma)
jamENG   = ql.JamshidianSwaptionEngine(hwMDL)

# ボラティリティ(stYR, swpTNR, nmVOL)
#volDATA = [(1,5,32.78), (2,4,30.32), (3,3,29.93), (4,2,30.30), (5,1,31.59)]
volDATA  = [(1,5,36.06)]  

# スワップション (ノーマルモデル: errorType, strike, nominal, volType)
nmlPRM   = (ql.BlackCalibrationHelper.RelativePriceError, 
                                            ql.nullDouble(), 1.0, ql.Normal)
swptnHLP = [ql.SwaptionHelper(ql.Period(stY, ql.Years),ql.Period(tnr, ql.Years),
            mu.sqHDL(nV/10000), tbrIX, pdFreqSA, dcA365, dcA365, tbCrvHDL,*nmlPRM ) 
            for (stY, tnr, nV) in volDATA ] 
for hh in swptnHLP: hh.setPricingEngine(jamENG) #スワップションにエンジン設定

# sigmaのみカリブレーション, aaは0.03に固定 ->> cnstPRM: constraint parameters
smplxMTD = ql.Simplex(0.05) ; endCRT = ql.EndCriteria(10000,100,1e-6,1e-8,1e-8)
cnstPRM  = (ql.NoConstraint(), [], [True, False])       #<< [fix aa, calib sigma]
hwMDL.calibrate(swptnHLP, smplxMTD, endCRT, *cnstPRM)

# カリブレーション結果      (NormalとBlackでfaceが異なる?)
print(f"カリブレーション結果: [aa, sigma] = {hwMDL.params()}")
print("スワップションヘルパー評価 ", end='' )
print(   f"HW: {swptnHLP[0].modelValue():.5%}", end=', ' )
print(f"Black: {swptnHLP[0].blackPrice(volDATA[0][2]):.3f}", end=', ')
print('インプライドvol: {:.6f}'.format(
    swptnHLP[0].impliedVolatility(swptnHLP[0].modelValue(), 1e-5, 50, 0.0, 0.50)))

カリブレーション結果: [aa, sigma] = [ 0.03; 0.00396173 ]
スワップションヘルパー評価 HW: 0.71682%, Black: 71.673, インプライドvol: 0.003606


**jamENGによるスワップション評価**  
- カリブ後のsigma=0.00396173で計算

In [3]:
# 原資産オブジェクトswapOBJの作成とエンジン設定
swEffDT,                 swMatDT,    ntlAMT,     cpnRT,   sprd,       payRcv     =\
mu.jDT(2023,8,23),mu.jDT(2028,8,23),10_000_000, 0.003579, 0.0, ql.VanillaSwap.Payer

fixSCD = ql.Schedule(swEffDT, swMatDT, pdFreqSA, calJP, mFLLW, mFLLW, dgRULEb, EoMf)
fltSCD = ql.Schedule(swEffDT, swMatDT, pdFreqSA, calJP, mFLLW, mFLLW, dgRULEb, EoMf)
swapOBJ = ql.VanillaSwap(payRcv, ntlAMT, fixSCD, cpnRT,       dcA365,
                                         fltSCD, tbrIX, sprd, dcA365)
swapOBJ.setPricingEngine(ql.DiscountingSwapEngine(tbCrvHDL))

# スワップションswptnOBJの評価 
opMatDT = mu.jDT(2023,8,21)
swptnOBJ = ql.Swaption(swapOBJ, ql.EuropeanExercise(opMatDT))
swptnOBJ.setPricingEngine(jamENG)
print(f'HW-sigma: {hwMDL.params()[1]:.3%}, NPV: {swptnOBJ.NPV():,.3f}')

HW-sigma: 0.396%, NPV: 71,484.141


<img src='TiborSwaptionHW-1x5.png' width='600'>